# **Cuarto conjunto de tareas a realizar**

## Paquetes necesarios e inicializaciones

La siguiente práctica consta dos partes principales, la primera de ellas basada en YOLO y en detección de matrículas personas y vehículos y la segunda en OCR (Optical Character Recognition).

Instalar labelme en el sistema operativo del anfitrión (NO EN EL ENVIRONMENT)

Hacer lo siguiente desde el CMD del sistema para pasar las imágenes en formato JSON obtenidas con labelme y pasarlas a formato YOLO.

Si se tiene una tarjeta gráfica de NVIDIA se puede utilizar la GPU haciendo uso de CUDA, para instalar CUDAv11.6 hacer uso del siguiente script.

In [5]:
import cv2
import math
import yaml
import csv
from collections import defaultdict
import numpy as np
from ultralytics import YOLO

In [6]:
# === ENTRENAMIENTO DEL MODELO ===
print("Iniciando entrenamiento del modelo")

# Carga el modelo base (preentrenado)
model = YOLO("yolo11l.pt")

# Entrena con tu dataset y configuración
results = model.train(
    data="data.yaml",
    imgsz=640,
    epochs=300,
    project="runs/train_custom",
    name="exp2",
    exist_ok=True,
    plots=True
)

print("Entrenamiento completado.")

Iniciando entrenamiento del modelo
Ultralytics 8.3.223  Python-3.9.23 torch-1.12.1 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
WARNING Upgrade to torch>=2.0.0 for deterministic training.
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11l.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=exp2, nbs=64, nms=False, opset=None, optimize=False,

In [7]:
print("CÁLCULO MANUAL DESDE MATRIZ DE CONFUSIÓN:")

# 1. Obtén la matriz de confusión (es un array de NumPy)
matrix = results.confusion_matrix.matrix

if matrix is not None and matrix.size > 0:
    # 2. Calcula la suma de la diagonal (aciertos correctos)
    # np.diag() extrae la diagonal
    aciertos_correctos = np.diag(matrix).sum()

    # 3. Calcula el total de predicciones (suma de toda la matriz)
    total_predicciones = matrix.sum()

    # 4. Calcula el acierto
    if total_predicciones > 0:
        accuracy_manual = aciertos_correctos / total_predicciones
        
        print(f"Matriz de Confusión: \n{matrix}")
        print(f"Aciertos (Diagonal): {aciertos_correctos}")
        print(f"Total de Muestras: {total_predicciones}")
        print(f"Porcentaje de Acierto (Accuracy Manual): {accuracy_manual * 100:.2f}%")
    else:
        print("La matriz de confusión está vacía.")
else:
    print("No se generó matriz de confusión (quizás no es un modelo de detección/clasificación).")

CÁLCULO MANUAL DESDE MATRIZ DE CONFUSIÓN:
Matriz de Confusión: 
[[         18           2]
 [          2           0]]
Aciertos (Diagonal): 18.0
Total de Muestras: 22.0
Porcentaje de Acierto (Accuracy Manual): 81.82%


In [8]:
import cv2
import csv
import numpy as np
from ultralytics import YOLO
import pytesseract
from PIL import Image

# Configurar la ruta de Tesseract (Solo necesario si no está en tu PATH de Windows)
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# ============================
# Modelos
# ============================
# ⚠️ CAMBIA ESTAS RUTAS A TUS ARCHIVOS
custom_model = YOLO("runs/train_custom/exp2/weights/best.pt") # Modelo de detección de matrículas
coco_model = YOLO("yolo11l.pt") # COCO preentrenado (Vehículos y personas)

vehicle_class_indices = [2, 3, 5, 7] # car, motorcycle, bus, truck
person_class_index = 0
coco_class_indices = [person_class_index] + vehicle_class_indices

# ============================
# Funciones de Reconocimiento de Matrículas
# ============================

def recognize_plate_ocr(plate_image):
    """Procesa la imagen de la matrícula usando Tesseract OCR."""
    try:
        # 1. Preprocesamiento básico
        plate_image_rgb = cv2.cvtColor(plate_image, cv2.COLOR_BGR2RGB)
        pil_image = Image.fromarray(plate_image_rgb)
        
        # 2. Configuración de Tesseract para matrículas (PSM 7: una sola línea)
        config = r'--oem 3 --psm 7 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
        
        text = pytesseract.image_to_string(pil_image, config=config).strip()
        # 3. Limpieza: eliminar espacios y caracteres no deseados
        return "".join(filter(str.isalnum, text)).upper()
    except Exception:
        return "OCR_ERROR"

def recognize_plate_vlm_mock(plate_image):
    """
    Simulación de reconocimiento con VLM (Vision-Language Model).
    Aquí se simula la llamada a un modelo VLM real.
    """
    # ⚠️ En un entorno real, aquí usarías un modelo VLM o un OCR avanzado basado en DL (como PaddleOCR/TrOCR)
    # Por ahora, devolvemos un texto constante para la comparación.
    
    # Simulación de un resultado con un algoritmo diferente:
    
    # -----------------------------------------------------------
    # Lógica de SIMULACIÓN (solo para ver la columna en el CSV)
    # -----------------------------------------------------------
    try:
        # Simulación de un mejor post-procesamiento o un modelo más robusto
        text_ocr = recognize_plate_ocr(plate_image)
        if len(text_ocr) == 7:
            return f"VLM-{text_ocr[0:4]}-OK" # Simular un acierto
        else:
            return "VLM-Simulado"
    except Exception:
        return "VLM_ERROR"
    # -----------------------------------------------------------


# ============================
# Tracker simple
# ============================
object_id_counter = 0
prev_centroids = {}

def get_centroid(box):
    x1, y1, x2, y2 = map(int, box)
    return int((x1+x2)/2), int((y1+y2)/2)

def assign_id(cx, cy, prev_centroids, threshold=50):
    global object_id_counter
    # Buscar el objeto más cercano
    for oid, (pcx, pcy) in prev_centroids.items():
        if np.sqrt((cx-pcx)**2 + (cy-pcy)**2) < threshold:
            prev_centroids[oid] = (cx, cy)
            return oid
    # Si no se encuentra, asignar nuevo ID
    object_id_counter += 1
    prev_centroids[object_id_counter] = (cx, cy)
    return object_id_counter

# ============================
# Dibujar bounding boxes
# ============================
def draw_boxes(results, frame, color=(0,255,0), filter_classes=None, text_data=None):
    for r in results:
        boxes = r.boxes
        for i, (box, cls, conf) in enumerate(zip(boxes.xyxy, boxes.cls, boxes.conf)):
            cls = int(cls)
            if filter_classes and cls not in filter_classes:
                continue
            x1, y1, x2, y2 = map(int, box)
            
            label = f"{r.names[cls]} {conf:.2f}"
            
            # Si hay datos de texto, añadirlo a la etiqueta
            if text_data and i < len(text_data):
                 ocr_text, vlm_text = text_data[i]
                 label += f" | OCR: {ocr_text} | VLM: {vlm_text}"

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, label, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    return frame

# ============================
# Función para verificar si la matrícula está dentro de un vehículo
# ============================
def plate_inside_vehicle(plate_box, vehicle_boxes):
    # Añadimos un pequeño margen para robustez
    margin = 5
    px1, py1, px2, py2 = plate_box
    for vx1, vy1, vx2, vy2 in vehicle_boxes:
        if (px1 >= vx1 + margin and 
            py1 >= vy1 + margin and 
            px2 <= vx2 - margin and 
            py2 <= vy2 - margin):
            return True
    return False

# ============================
# Video
# ============================
video_path = "./Resources/video.mp4" # ⚠️ REEMPLAZA ESTA RUTA
cap = cv2.VideoCapture(video_path)

cv2.namedWindow("Detections", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Detections", 960, 540)

# ============================
# CSV
# ============================
csv_file = "detecciones_comparadas.csv"
csv_headers = [
    "fotograma", "tipo_objeto", "confianza", "id_tracking", 
    "x1", "y1", "x2", "y2",
    "matricula_en_su_caso", "conf_matricula", "mx1","my1","mx2","my2", 
    "texto_OCR", "texto_VLM" # Columnas para la comparación
]

f = open(csv_file, mode='w', newline='', encoding='utf-8')
writer = csv.writer(f)
writer.writerow(csv_headers)

# ============================
# Procesar frames
# ============================
frame_num = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_num += 1

    # Detecciones de COCO
    coco_results = coco_model.predict(frame, imgsz=640, conf=0.25, verbose=False)
    vehicle_boxes_current = []
    
    # Procesar resultados de COCO y Tracking
    for r in coco_results:
        boxes = r.boxes
        for box, cls, conf in zip(boxes.xyxy, boxes.cls, boxes.conf):
            cls = int(cls)
            if cls not in coco_class_indices:
                continue
            x1, y1, x2, y2 = map(int, box)
            cx, cy = get_centroid((x1, y1, x2, y2))
            oid = assign_id(cx, cy, prev_centroids)
            tipo_obj = r.names[cls]
            
            # Escribir la detección del vehículo/persona
            writer.writerow([frame_num, tipo_obj, float(conf), oid, x1, y1, x2, y2,
                              "", "", "", "", "", "", "", ""])
            
            if cls in vehicle_class_indices:
                vehicle_boxes_current.append((x1, y1, x2, y2))

    # Detecciones MATRÍCULAS
    custom_results = custom_model.predict(frame, imgsz=640, conf=0.25, verbose=False)
    plate_text_data = [] # Para pasar el texto al dibujado

    for r in custom_results:
        boxes = r.boxes
        for box, cls, conf in zip(boxes.xyxy, boxes.cls, boxes.conf):
            x1, y1, x2, y2 = map(int, box)
            
            # Solo si está dentro de un vehículo detectado
            if plate_inside_vehicle((x1, y1, x2, y2), vehicle_boxes_current):
                cx, cy = get_centroid((x1, y1, x2, y2))
                oid = assign_id(cx, cy, prev_centroids)
                
                # Recortar la imagen de la matrícula
                plate_crop = frame[y1:y2, x1:x2]
                
                # Reconocimiento por OCR y VLM
                texto_ocr = recognize_plate_ocr(plate_crop)
                texto_vlm = recognize_plate_vlm_mock(plate_crop)

                plate_text_data.append((texto_ocr, texto_vlm))

                # Escribir la detección de la matrícula y los resultados
                writer.writerow([
                    frame_num, r.names[int(cls)], float(conf), oid, x1, y1, x2, y2,
                    "plate", float(conf), x1, y1, x2, y2, 
                    texto_ocr, # Resultado OCR
                    texto_vlm  # Resultado VLM (simulado)
                ])

    # ============================
    # Dibujar bounding boxes
    # ============================
    # Dibujar COCO (Vehículos/Personas)
    frame = draw_boxes(coco_results, frame, color=(0,255,0), filter_classes=coco_class_indices)
    # Dibujar MATRÍCULAS (Rojo) y mostrar el texto OCR/VLM en la etiqueta
    frame = draw_boxes(custom_results, frame, color=(0,0,255), text_data=plate_text_data) 

    display_frame = cv2.resize(frame, (960, 540))
    cv2.imshow("Detections", display_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
f.close()
cv2.destroyAllWindows()
print(f"CSV guardado en {csv_file}")

CSV guardado en detecciones_comparadas.csv
